# Import kubeflow pipeline libraries


In [44]:
import kfp
import kfp.components as comp
import kfp.dsl as dsl
from kfp.components import InputPath, OutputPath

In [45]:
from kfp.components import func_to_container_op

## Download Dataset

In [46]:
#Download data
def download_data(download_link: str, data_path: OutputPath(str)):
    import zipfile
    import wget
    import os

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    # download files
    wget.download(download_link.format(file='train'), f'{data_path}/train_csv.zip')
    wget.download(download_link.format(file='test'), f'{data_path}/test_csv.zip')
    
    with zipfile.ZipFile(f"{data_path}/train_csv.zip","r") as zip_ref:
        zip_ref.extractall(data_path)
        
    with zipfile.ZipFile(f"{data_path}/test_csv.zip","r") as zip_ref:
        zip_ref.extractall(data_path)
    
    return(print('Done!'))

In [47]:
download_op = func_to_container_op(download_data,base_image="python:3.7", packages_to_install=['wget', 'zipfile36'])

# Load Data

In [48]:
def load_data(data_path:comp.InputPath(str),load_data_path: comp.OutputPath()):
    import pandas as pd
    import os, pickle
    
    train_data_path = data_path + '/train.csv'
    test_data_path = data_path + '/test.csv'
    tweet_df= pd.read_csv(train_data_path)
    test_df=pd.read_csv(test_data_path)
    df=pd.concat([tweet_df,test_df])
    
    #creating the preprocess directory
    os.makedirs(load_data_path, exist_ok = True)
    
    # join train and test together
    ntrain = tweet_df.shape[0]
    ntest = test_df.shape[0]
    with open(f'{load_data_path}/df', 'wb') as f:
        pickle.dump((ntrain, df, tweet_df), f)
    return(print('Done!'))

In [49]:
load_data_step = func_to_container_op(
    load_data,
    output_component_file='load_data_component.yaml', # This is optional. It saves the component spec for future use.
    base_image='python:3.7',
    packages_to_install=['pandas','pickle5'])

In [50]:
def preprocess_data(load_data_path:comp.InputPath(str), preprocess_data_path:comp.OutputPath(str)):
    
    import re
    import pandas as pd
    import os, pickle
    import string
    #import spacy
    #import gensim.downloader as api
    
    #nlp = spacy.load("en_core_web_sm")
    #wv = api.load('word2vec-google-news-300')
    
     #loading the train data
    with open(f'{load_data_path}/df', 'rb') as f:
        ntrain, df, tweet_df = pickle.load(f)
        
    def clean_text(text):
        
        '''
            This method generates a clean text and returns a dataframe
        '''
        text = re.sub(r'(\d{3})-(\d{3})-(\d{4})',' ', text)
        text = re.sub(r'(\d{2})/(\d{2})',' ', text)
        text = re.sub(r'\S*@\S*\s?',' ', text)
        text = re.sub(r'\s+',' ', text)
        text = re.sub(r'\'',' ', text)
        text = re.sub(r'<.*?>',' ', text)
        text = re.sub(r'^https?:\/\/.*[\r\n]*',' ', text)
        text = re.sub(r'[^0-9a-z #+_]',' ', text)
        text = re.sub(r'\(.*?\)',' ', text)
        text = re.sub(r'(\w+\.\w+\.com)',' ', text)
        text = re.sub(r'\w+\d+\w+|\w+\d+|\d+\w+',' ', text)
        text = re.sub(r'\d+|\@|\*|\=|\+|\$|\%|\.\.|\#|-|\[|\]|\?|\"',' ', text)
        text = re.sub(r'[ \n]+',' ', text)
        return text.strip().lower()
    
    # def preprocess(text):
    #     doc = nlp(text)
    #     filtered_token = []
    #     for token in doc:
    #         if token.is_stop or token.is_punct:
    #             continue
    #         filtered_token.append(token.lemma_)
    #     return wv.get_mean_vector(filtered_token)

    
    df['text'] = df['Summary'].apply(clean_text)
    #df['text'] = df['cleaned_txt'].apply(preprocess)
    
   
    #creating the preprocess directory
    os.makedirs(preprocess_data_path, exist_ok = True)
    
    with open(f'{preprocess_data_path}/df', 'wb') as f:
        pickle.dump((ntrain, df, tweet_df), f)
    return(print('Done!'))

In [51]:
preprocess_data_step = func_to_container_op(preprocess_data, packages_to_install=['pandas', 'regex', 'pickle5'], output_component_file='preprocess_data_component.yaml')

In [52]:
def corpus_creation(preprocess_data_path: comp.InputPath(str), corpus_path: comp.OutputPath(str)):
    import os, pickle
    import pandas as pd
    import nltk
    nltk.download('stopwords')
    nltk.download('punkt')
    from nltk.corpus import stopwords
    from nltk.util import ngrams
    from nltk.tokenize import word_tokenize
    stop=set(stopwords.words('english'))
    from tqdm import tqdm
    
    with open(f'{preprocess_data_path}/df', 'rb') as f:
        ntrain, df, tweet_df = pickle.load(f)
        
    def create_corpus(df):
        corpus=[]
        for tweet in tqdm(df['text']):
            words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
            corpus.append(words)
        return corpus
    
     #creating the preprocess directory
    os.makedirs(corpus_path, exist_ok = True)
    
    corpus=create_corpus(df)
    with open(f'{corpus_path}/corpus', 'wb') as f:
        pickle.dump((corpus,tweet_df), f)
    return(print('Done!'))

In [53]:
corpus_creation_step = func_to_container_op(corpus_creation, packages_to_install=['pandas', 'pickle5', 'nltk','tqdm'], output_component_file='corpus_creation_component.yaml')

In [54]:
def embedding_step(download_link_glove: str,corpus_path: comp.InputPath(str), embedded_path: comp.OutputPath(str)):
    
    import os, pickle
    import pandas as pd
    import zipfile
    import wget
    import os
    from keras.preprocessing.text import Tokenizer
    from keras.utils import pad_sequences
    import numpy as np
    from tqdm import tqdm
    
    with open(f'{corpus_path}/corpus', 'rb') as f:
        corpus, tweet_df = pickle.load(f)
    
    if not os.path.exists(embedded_path):
        os.makedirs(embedded_path)
    # download files
    wget.download(download_link_glove, f'{embedded_path}/glove.6B.zip')
    
    with zipfile.ZipFile(f'{embedded_path}/glove.6B.zip', 'r') as zip_ref:
        zip_ref.extractall(embedded_path)
    
    embedding_dict={}
    """path_to_glove_file = os.path.join(
        os.path.expanduser("~"), f"{embedded_path}/glove.6B.100d.txt"
    )"""
    with open(f"{embedded_path}/glove.6B.100d.txt",'r') as f:
        for line in f:
            values=line.split()
            word=values[0]
            vectors=np.asarray(values[1:],'float32')
            embedding_dict[word]=vectors
    f.close()
    
    MAX_NB_WORDS = 50000
    MAX_LEN=250
    tokenizer_obj=Tokenizer(num_words=MAX_NB_WORDS, lower=True)
    tokenizer_obj.fit_on_texts(corpus)
    sequences=tokenizer_obj.texts_to_sequences(corpus)
    
    tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')
    word_index=tokenizer_obj.word_index
    num_words=len(word_index)+1
    embedding_matrix=np.zeros((num_words,100))

    for word,i in tqdm(word_index.items()):
        if i > num_words:
            continue

        emb_vec=embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i]=emb_vec
    
    with open(f'{embedded_path}/embedding', 'wb') as f:
        pickle.dump((embedding_matrix, num_words, tweet_pad, tweet_df, MAX_LEN), f)
    return(print('Done!'))

In [55]:
embedding_creation_step = func_to_container_op(embedding_step, packages_to_install=['pandas', 'zipfile36', 'wget','tqdm','keras','numpy','tensorflow', 'pickle5'], output_component_file='embedding_component.yaml')

In [56]:
def model_building_and_training(embedded_path: comp.InputPath(str), model_path: comp.OutputPath(str)):
    
    import os, pickle;
    import pandas as pd
    import numpy as np
    from keras.models import Sequential
    from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
    from keras.initializers import Constant
    from sklearn.model_selection import train_test_split
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping 
    
    with open(f'{embedded_path}/embedding', 'rb') as f:
        embedding_matrix, num_words, tweet_pad, tweet_df, MAX_LEN = pickle.load(f)
    
    train=tweet_pad[:tweet_df.shape[0]]
    final_test=tweet_pad[tweet_df.shape[0]:]
    X_train,X_test,y_train,y_test=train_test_split(train,tweet_df['label_num'].values,test_size=0.2)
    
    #defining model
    model=Sequential()

    embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                       input_length=MAX_LEN,trainable=False)

    model.add(embedding)
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(128, return_sequences=True,dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(64,dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(10, activation='softmax'))
    
    
    #Compiling the classifier model with Adam optimizer
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    #fitting the model
    history=model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test),callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)],verbose=2)

    #evaluate model
    test_loss, test_acc = model.evaluate(np.array(X_test),  np.array(y_test), verbose=0)
    print("Test_loss: {}, Test_accuracy: {} ".format(test_loss,test_acc))
    
    #creating the preprocess directory
    os.makedirs(model_path, exist_ok = True)
    
    #saving the model
    model.save(f'{model_path}/model.h5')
    
    #dumping other values
    with open(f'{model_path}/values', 'wb') as f:
        pickle.dump((test_loss, test_acc), f)
    return(print('Done!'))

In [57]:
model_building_step = func_to_container_op(model_building_and_training, packages_to_install=['pandas', 'zipfile36', 'wget','tqdm','keras','numpy','tensorflow','scikit-learn','pickle5'], output_component_file='model_creation_component.yaml')

In [66]:
from kfp import dsl

@dsl.pipeline(
    name='ITCE pipeline',
    description='A simple pipeline for testing ITC Engine.',
)
def my_pipeline(
    download_link: str = 'https://github.com/nktyagi123/kubeflow_itce/blob/main/{file}.csv.zip?raw=true',
    data_path: str = "/mnt",
    load_data_path: str = "load", 
    preprocess_data_path: str = "preprocess",
    corpus_path: str = "corpus",
    download_link_glove: str = "http://nlp.stanford.edu/data/glove.6B.zip",
    model_path: str = "model",
):
    download_container = download_op(download_link)
    output1 = load_data_step(download_container.output)
    output2 = preprocess_data_step(output1.output)
    output3 = corpus_creation_step(output2.output)
    output4 = embedding_creation_step(download_link_glove, output3.output)
    output5 = model_building_step(output4.output)

# Compile the pipeline
kfp.compiler.Compiler().compile(my_pipeline, 'ITCE_Complete.yaml')



In [62]:
import kfp
from kfp import dsl
# replace download_link with the repo link where the data is stored https:github-repo/data-dir/{file}.csv.zip?raw=true
download_link = 'https://github.com/nktyagi123/kubeflow_itce/blob/main/{file}.csv.zip?raw=true'
data_path = "/mnt"
load_data_path = "load"
preprocess_data_path = "preprocess"
corpus_path = "corpus",
download_link_glove = "http://nlp.stanford.edu/data/glove.6B.zip"
model_path = "model"

load_data_op = kfp.components.load_component_from_file('load_data_component.yaml')
preprocess_average_op = kfp.components.load_component_from_file('preprocess_data_component.yaml')
#save_result_op = kfp.components.load_component_from_file('save_result_component.yaml')



In [67]:
client = kfp.Client()
arguments = {"download_link": download_link,
             "data_path": data_path,
             "load_data_path": load_data_path,
             "preprocess_data_path": preprocess_data_path,
             "corpus_path":corpus_path,
             "download_link_glove":download_link_glove,
             "model_path":model_path,
            }
client.create_run_from_pipeline_func(trial_pipeline, arguments=arguments) 

IndentationError: unexpected indent (39024955.py, line 2)